In [1]:
import os
import cv2
import numpy as np
from skimage import exposure
from skimage.restoration import denoise_nl_means
from skimage.transform import resize
from albumentations import (
    Compose,
    HorizontalFlip,
    VerticalFlip,
    RandomRotate90,
    GaussNoise
)

In [2]:
# Step 1: Image resizing and normalization
def resize_and_normalize(image, desired_width, desired_height):
    resized_image = resize(image, (desired_height, desired_width))
    normalized_image = exposure.rescale_intensity(resized_image, out_range=(0, 255))
    return normalized_image

In [3]:
# Step 2: Image augmentation
def augment_image(image):
    augmentation_transform = Compose([
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        RandomRotate90(p=0.5),
        GaussNoise(p=0.2)
    ])
    augmented_image = augmentation_transform(image=image)["image"]
    return augmented_image

In [4]:
# Step 3: Noise reduction
def reduce_noise(image):
    denoised_image = denoise_nl_means(image, patch_size=5, patch_distance=3, h=0.8)
    return denoised_image

In [5]:
# # Step 4: Contrast enhancement
# def enhance_contrast(image, alpha=1.5, beta=0):
#     enhanced_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
#     return enhanced_image

In [6]:
# Step 5: ROI extraction (Example: Keep only the center region)
def extract_roi(image):
    height, width = image.shape[:2]
    roi_width = int(width * 0.8)
    roi_height = int(height * 0.9)
    x = int((width - roi_width) / 2)
    y = int((height - roi_height) / 2)
    roi = image[y:y+roi_height, x:x+roi_width]
    return roi

In [7]:
# Path to the folder containing the raw PNG images
folder_path = "C:/AI PROJECT 2/Dataset/boneage-training-dataset-raw/boneage-training-dataset"

# Output folder for preprocessed images
output_folder = "C:/AI PROJECT 2/Dataset/boneage-training-dataset"

# Desired dimensions for resizing
desired_width = 256
desired_height = 256

In [8]:
# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Step 1: Image resizing and normalization
        resized_normalized_image = resize_and_normalize(image, desired_width, desired_height)

        # Step 2: Image augmentation
        augmented_image = augment_image(resized_normalized_image)

        # Step 3: Noise reduction
        denoised_image = reduce_noise(augmented_image)

        # Step 4: Contrast enhancement
        contrast_enhanced_image = enhance_contrast(denoised_image)

        # Step 5: ROI extraction
        roi = extract_roi(contrast_enhanced_image)

        # Save the preprocessed image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, roi)